In [13]:
import sys
sys.path.insert(1, '/Users/mac/Desktop/PycharmProjects/TAADL/src')
sys.path.insert(2, '/Users/mac/Desktop/PycharmProjects/TAADL/models')

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import json

from torch import optim
from param_search import train
from network import GPCopulaRNN
from config import DATA_PATH, e_dict
from utils import transform, inv_transform

In [2]:
#TODO add mean and covariance to existing RP strategy and do the plotting

In [16]:
params = torch.load('/Users/mac/Desktop/PycharmProjects/TAADL/models/GaussCopula.pt')

In [3]:
# import feature 
prices = pd.read_csv(DATA_PATH+'/prices.csv', index_col='Date')

# preprocess weekly data
prices.index = pd.to_datetime(prices.index)
prices = prices.resample('M').last()
lret_m = np.log(prices/prices.shift(1)).fillna(0.0) # weekly return


# convert the data to torch tensor
split_idx = int(lret_m.shape[0] * 0.7)
Z_tr, Z_te = torch.Tensor(lret_m.iloc[:split_idx].values), torch.Tensor(lret_m.iloc[split_idx:].values)
X_tr, cdf = transform(Z_tr)
X_te, _   = transform(Z_te, cdf)


In [4]:
num_samples = 20
num_epochs = 250

# load parameters for model and optimizer
model = GPCopulaRNN(input_size=1, hidden_size=4, num_layers=2, rank_size=4, 
                        batch_size=3, num_asset=7, dropout=0.05, features=e_dict)

optimizer = optim.Adam(params=model.parameters(), lr=params['learning rate'], weight_decay=hyp_params['weight decay'])
model.load_state_dict(torch.load('/Users/mac/Desktop/PycharmProjects/TAADL/models/GaussCopula.json')['networks_params'])

<All keys matched successfully>

In [5]:
# re-train the model
train(model, Z_tr, optimizer, num_samples, num_epochs, save=False)

Epoch 1
Gaussian LL Loss : -20.670000076293945
Validation MSE   : 0.01258373074233532 

Epoch 51
Gaussian LL Loss : -22.549999237060547
Validation MSE   : 0.01209927350282669 

Epoch 101
Gaussian LL Loss : -26.459999084472656
Validation MSE   : 0.011848720721900463 

Epoch 151
Gaussian LL Loss : -28.209999084472656
Validation MSE   : 0.011558487080037594 

Epoch 201
Gaussian LL Loss : -30.770000457763672
Validation MSE   : 0.01137320976704359 



tensor(0.0112)

In [55]:
def test(model, data, cdfs, num_samples=None):
    loss = torch.Tensor([0.0])
    count = 0
    for i in range(6,data.size(0)):
        z, z_targ = data[i-6:i], data[i:i+1]    
        count += 1
        with torch.no_grad():
            # run the model
            mu_t, _, _ = model(z, pred=True)
            z_hat = inv_transform(mu_t[-1], cdfs)
        
            # use sampled z hat for next step prediction
            mu_pred, _, _ = model(z_hat.unsqueeze(0), pred=True)
            Z_tr_hat = inv_transform(mu_pred.detach(), cdfs)
        
        loss = loss + (Z_tr_hat - z_targ).pow(2).sum()
    loss = loss/count
    print(f'Prediction MSE : {loss}')
    return loss

In [56]:
test(model, Z_te, cdf)

tensor([[-0.2954],
        [-0.2889],
        [-0.2881],
        [-0.2927],
        [-0.2716],
        [-0.2859],
        [-0.2968]])
tensor([[-0.0097],
        [-0.0091],
        [-0.0090],
        [-0.0095],
        [-0.0068],
        [-0.0088],
        [-0.0099]])
tensor([[-0.2954],
        [-0.2889],
        [-0.2881],
        [-0.2927],
        [-0.2717],
        [-0.2859],
        [-0.2969]])
tensor([[-0.0097],
        [-0.0091],
        [-0.0090],
        [-0.0095],
        [-0.0069],
        [-0.0088],
        [-0.0099]])
tensor([[-0.2954],
        [-0.2889],
        [-0.2879],
        [-0.2927],
        [-0.2717],
        [-0.2859],
        [-0.2969]])
tensor([[-0.0097],
        [-0.0091],
        [-0.0090],
        [-0.0095],
        [-0.0069],
        [-0.0088],
        [-0.0099]])
tensor([[-0.2954],
        [-0.2889],
        [-0.2881],
        [-0.2927],
        [-0.2716],
        [-0.2859],
        [-0.2968]])
tensor([[-0.0097],
        [-0.0091],
        [-0.0090],
     

tensor([0.0124])